In [25]:

import chess
import chess.pgn
import chess.engine
import random
import time
from math import log,sqrt,e,inf

class node:
    def __init__(self,*board):
        if(len(board)>0):
            self.state = board
        else:
            self.state = chess.Board()
        self.action = ''
        self.children = set()
        self.parent = None
        self.N = 0
        self.n = 0
        self.v = 0

def ucb1(curr_node):
    ans = curr_node.v+2*(sqrt(log(curr_node.N+e+(10**-6))/(curr_node.n+(10**-10))))
    return ans

def rollout(board,curr_node):
    
    if(curr_node.state.is_game_over()):
        board1 = curr_node.state
        if(board1.result()=='1-0'):
            #print("h1")
            return (1,curr_node)
        elif(board1.result()=='0-1'):
            #print("h2")
            return (-1,curr_node)
        else:
            return (0.5,curr_node)
    
    all_moves = [curr_node.state.san(i) for i in list(curr_node.state.legal_moves)]
    
    for i in all_moves:
        tmp_state = chess.Board(curr_node.state.fen())
        tmp_state.push_san(i)
        child = node(board)
        child.state = tmp_state
        child.parent = curr_node
        curr_node.children.add(child)
    rnd_state = random.choice(list(curr_node.children))

    return rollout(board,rnd_state)

def expand(curr_node,white):
    if(len(curr_node.children)==0):
        return curr_node
    max_ucb = -inf
    if(white):
        idx = -1
        max_ucb = -inf
        sel_child = None
        for i in curr_node.children:
            tmp = ucb1(i)
            if(tmp>max_ucb):
                idx = i
                max_ucb = tmp
                sel_child = i

        return(expand(sel_child,0))

    else:
        idx = -1
        min_ucb = inf
        sel_child = None
        for i in curr_node.children:
            tmp = ucb1(i)
            if(tmp<min_ucb):
                idx = i
                min_ucb = tmp
                sel_child = i

        return expand(sel_child,1)

def rollback(curr_node,reward):
    curr_node.n+=1
    curr_node.v+=reward
    while(curr_node.parent!=None):
        curr_node.N+=1
        curr_node = curr_node.parent
    return curr_node

def mcts_pred(board,curr_node,over,white,choice,iterations=10):
    if(over):
        return -1
    all_moves = [curr_node.state.san(i) for i in list(curr_node.state.legal_moves)]
    map_state_move = dict()
    
    for i in all_moves:
        tmp_state = chess.Board(curr_node.state.fen())
        tmp_state.push_san(i)
        child = node(board)
        child.state = tmp_state
        child.parent = curr_node
        curr_node.children.add(child)
        map_state_move[child] = i
        
    while(iterations>0):
        if(white):
            idx = -1
            max_ucb = -inf
            sel_child = None
            for i in curr_node.children:
                tmp = ucb1(i)
                if(tmp>max_ucb):
                    idx = i
                    max_ucb = tmp
                    sel_child = i
            ex_child = expand(sel_child,0)
            reward,state = rollout(board,ex_child)
            curr_node = rollback(state,reward)
            iterations-=1
        elif(choice==1):
            idx = -1
            min_ucb = inf
            sel_child = None
            for i in curr_node.children:
                tmp = ucb1(i)
                if(tmp<min_ucb):
                    idx = i
                    min_ucb = tmp
                    sel_child = i

            ex_child = expand(sel_child,1)

            reward,state = rollout(board,ex_child)

            curr_node = rollback(state,reward)
            iterations-=1
    if(white):
        
        mx = -inf
        idx = -1
        selected_move = ''
        for i in (curr_node.children):
            tmp = ucb1(i)
            if(tmp>mx):
                mx = tmp
                selected_move = map_state_move[i]
        return selected_move
    elif(choice==1):
        mn = inf
        idx = -1
        selected_move = ''
        for i in (curr_node.children):
            tmp = ucb1(i)
            if(tmp<mn):
                mn = tmp
                selected_move = map_state_move[i]
        return selected_move

#to randomize the starting position we can customize the board. for now I am using a random board setup
#board = chess.Board("r1bqkb1r/pppp1Qpp/2n2n2/4p3/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 0 4")

#to let the pc choose random positions
board = chess.Board.from_chess960_pos(random.randint(0, 959))
#board = chess.Board()
print(board)
choice=int(input("Do you want 1. AI vs AI game or 2.Human vs AI game? write 1 or 2"))
if(choice==2):
    white=int(input("Do you want to play as 0.First player or 1.Second player? write 0 or 1"))
else:
    white = 1
moves = 0
pgn = []
game = chess.pgn.Game()
evaluations = []
sm = 0
j=1



while((not board.is_game_over())):
    all_moves = [board.san(i) for i in list(board.legal_moves)]
    start = time.time()
    if(white==1 or choice==1):
        root = node()
        root.state = board
        result = mcts_pred(board,root,board.is_game_over(),white,choice)
        print("time taken to iteration",j,(time.time()-start))
    elif(choice==2):
        print("input format should be something like a2, b6, g3, first one is row, second one is colum")
        print("column is a to h if you are second player, otherwise A to H will be your column and row is from 8 to 1 (up to down)")
        to=input("Write where you want to go")
        frm = input("write from where you want to move")
        result=frm+to
    try:
        board.push_san(result)
    except:
        print(result)
        print("invalid move. game over")
        break
    #print(result)
    pgn.append(result)
    white ^= 1
    j+=1
    
    moves+=1
    print(board)
    print("-----------move done-----------")
#print(board)
print(" ".join(pgn))
print()
#print(evaluations)
print(board.result())
game.headers["Result"] = board.result()


r n q k n b b r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N Q K N B B R
Do you want 1. AI-AI game or 2.Human-AI game? write 1 or 21
time taken to iteration 1 4.82654595375061
r n q k n b b r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . N . . . .
P P P P P P P P
R N Q K . B B R
-----------move done-----------
time taken to iteration 2 5.0598461627960205
r n q k n b b r
p p p p p . p p
. . . . . . . .
. . . . . p . .
. . . . . . . .
. . . N . . . .
P P P P P P P P
R N Q K . B B R
-----------move done-----------
time taken to iteration 3 5.964323997497559
r n q k n b b r
p p p p p . p p
. . . . . . . .
. . . . . p . .
P . . . . . . .
. . . N . . . .
. P P P P P P P
R N Q K . B B R
-----------move done-----------
time taken to iteration 4 5.056308031082153
r n q k n b b r
. p p p p . p p
p . . . . . . .
. . . . . p . .
P . . . . . . .
. . . N . . . .
. P P P P P P P
R N Q K . B B R
-----------move done--------

time taken to iteration 41 3.5725769996643066
. q . k . b B .
. . . . . . . r
. . p n . . . p
p p . . . . p .
P . . . p p P .
. P . . . P . .
n P R P P . . P
. . . Q K . B R
-----------move done-----------
time taken to iteration 42 3.612794876098633
. q . . . b B .
. . . k . . . r
. . p n . . . p
p p . . . . p .
P . . . p p P .
. P . . . P . .
n P R P P . . P
. . . Q K . B R
-----------move done-----------
time taken to iteration 43 3.4943671226501465
. q . . . b B .
. . . k . . . r
. . R n . . . p
p p . . . . p .
P . . . p p P .
. P . . . P . .
n P . P P . . P
. . . Q K . B R
-----------move done-----------
time taken to iteration 44 3.9127979278564453
. q . . . . B .
. . . k b . . r
. . R n . . . p
p p . . . . p .
P . . . p p P .
. P . . . P . .
n P . P P . . P
. . . Q K . B R
-----------move done-----------
time taken to iteration 45 3.2251577377319336
. q . . . . . .
. . . k b B . r
. . R n . . . p
p p . . . . p .
P . . . p p P .
. P . . . P . .
n P . P P . . P
. . . Q K . B R
---

time taken to iteration 81 3.044677972793579
. . . . . b . .
. . . r . q . .
. . k . P . . .
p . . . . . p p
p . . . . p P .
. P n B B p . .
. P . P . . . P
. . . Q K . . R
-----------move done-----------
time taken to iteration 82 3.338905096054077
. . . . . b . .
. . . r . . . .
. . k . P . . .
p . . . . q p p
p . . . . p P .
. P n B B p . .
. P . P . . . P
. . . Q K . . R
-----------move done-----------
time taken to iteration 83 2.997170925140381
. . . . . b . .
. . . r . . . .
B . k . P . . .
p . . . . q p p
p . . . . p P .
. P n . B p . .
. P . P . . . P
. . . Q K . . R
-----------move done-----------
time taken to iteration 84 3.150178909301758
. . . . . b . .
. . . r . . . .
B . k . P . . .
p . . n . q p p
p . . . . p P .
. P . . B p . .
. P . P . . . P
. . . Q K . . R
-----------move done-----------
time taken to iteration 85 3.121074914932251
. . . . . b . .
. . . r . . . .
. . k . P . . .
p . . n . q p p
p . . . . p P .
. P . . B p . .
. P . P . . . P
. . . Q K B . R
-------

time taken to iteration 121 1.74013090133667
R . . . . . . .
. . . q . . . .
. . . . . . . .
. . . k . . p p
. . . P . p . .
. p Q . n p . .
. p . . . . . P
. . . . K . . R
-----------move done-----------
time taken to iteration 122 1.5139811038970947
R . . . . . . .
. . . q . . . .
. . . . k . . .
. . . . . . p p
. . . P . p . .
. p Q . n p . .
. p . . . . . P
. . . . K . . R
-----------move done-----------
time taken to iteration 123 1.7376132011413574
R . . . . . . .
. . Q q . . . .
. . . . k . . .
. . . . . . p p
. . . P . p . .
. p . . n p . .
. p . . . . . P
. . . . K . . R
-----------move done-----------
time taken to iteration 124 2.2654569149017334
R . . . . . . .
. . Q . . . . .
. . . . k . . .
. . . . . . p p
. . . q . p . .
. p . . n p . .
. p . . . . . P
. . . . K . . R
-----------move done-----------
time taken to iteration 125 2.1753649711608887
. . . . . . . .
. . Q . . . . .
. . . . k . . .
. . . . . . p p
R . . q . p . .
. p . . n p . .
. p . . . . . P
. . . . K . . R